In [ ]:
import numpy as np
import os, glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,regularizers, losses
from tensorflow.keras.callbacks import LearningRateScheduler

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv3D, Flatten,LSTM,TimeDistributed,BatchNormalization,MaxPooling3D
from tensorflow.keras.layers import MaxPooling3D,Dropout,concatenate,AveragePooling2D, Reshape, Add, Conv2D
from tensorflow.keras.layers import UpSampling3D, UpSampling2D
import random
from tensorflow.keras.layers import Conv3DTranspose
import open3d as o3d
from scipy.io import loadmat
import scipy.io
# import pyvista as pv


In [ ]:
class CustomLoss_BCE(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        
        _epsilon = tf.convert_to_tensor(0.000001, y_pred.dtype.base_dtype)
        y_pred = tf.clip_by_value(y_pred, _epsilon, 1 - _epsilon)

        s1 = tf.math.subtract(1.0, y_true)
        s2 = tf.math.subtract(1.0, y_pred)
        
        a = tf.math.multiply_no_nan(tf.math.log(y_pred), y_true)
        b = tf.math.multiply_no_nan(tf.math.log(s2), s1)
        out = -(25*a + b)
        return out


In [ ]:
# #Input data

# folder_path = '/gpfs-volume/ITU-22/'
# c=0
# files_names1 = []
# for filename in glob.glob(os.path.join(folder_path, 'filesName.mat')):
#     files_names1.append(filename)

In [ ]:
# #Input data

# folder_path = ''
# c=0
# files_input1 = []
# for filename in glob.glob(os.path.join(folder_path, 'pCIR_area2_v2.mat')):
#     files_input1.append(filename)

# print(len(files_input1))
# print(files_input1[0])

In [ ]:
min_bound=[-16, -16, -2]
max_bound=[16,16,2]
voxel_size=0.25

x_lim = int((max_bound[0] - min_bound[0])/voxel_size);
y_lim = int((max_bound[1] - min_bound[1])/voxel_size);
z_lim = int((max_bound[2] - min_bound[2])/voxel_size);

print(x_lim, y_lim, z_lim)

In [ ]:
def convert2(x, y, z):
    
    x_val = (x - min_bound[0])/voxel_size
    y_val = (y - min_bound[1])/voxel_size
    z_val = (z - min_bound[2])/voxel_size
    
    return int(np.floor(x_val)), int(np.floor(y_val)), int(np.floor(z_val))

In [ ]:
# import mat73
# filename_inp = files_input1[0]
# with open(filename_inp, 'r') as f:
#     rf_signal = mat73.loadmat(f.name)['Pwr_RTP']
#     Xp_all_area2= rf_signal
# np.save("X_all_area3", Xp_all_area3)

In [ ]:
# # This code segment would be needed for training purpose
# # For generating the output data in appropriate format

# def load_data2(batch_size=1160):
    
#     Y_all = np.zeros((batch_size, x_lim, y_lim, z_lim))
#     aa = batch_size
   
#     filenamess = files_names1[0]
#     with open(filenamess, 'r') as f:
#         files = loadmat(f.name)
#         #print(files)
#         #print(files['filesName'][0])
        
#     for i in range(batch_size):
#         print(i, Y_all[i].shape)
#         filename_inp = files['filesName'][i]
#         #print(filename_inp)
#         filename_out = filename_inp
#         filename_out = filename_out.replace('mat', 'pcd')
#         filename_out = filename_out.replace('CIR_1', '/gpfs-volume/ITU_challenge_2/area1/lidar/LidarFrame')
        
#         with open(filename_out, 'r') as f:
#             pcd = o3d.io.read_point_cloud(f.name) 
#             out = np.asarray(pcd.points)
            
#             s = out.shape[0]
#             for ind in range(s):
#                 if(out[ind, 0] >= -16 and out[ind, 0] < 16 and out[ind, 1] >= -16 and out[ind, 1] < 16 and out[ind, 2] >= -2 and out[ind, 2] < 2):
#                     [x_val, y_val, z_val] = convert(out[ind, :]) 
#                     Y_all[i, x_val, y_val, z_val] = 1
# #             print(np.sum(Y_train[i, :,:,:]))
    
#    # min_val = np.min(X_all[:])           
#    # max_val = np.max(X_all[:])
#    # X_all = (X_all - min_val)/(max_val - min_val)
    
#     return Y_all

# Y_all=load_data2()
# np.save("Y_all_area3", Y_all)

In [ ]:
#Pre-processed input files for area 1 and area 3
Xp_all_area1 = np.load("X_all_area1.npy")
Xp_all_area3 = np.load("X_all_area3.npy")

In [ ]:
#output LIDAR PCD files converted to voxel grid for training purpose
Y_all_area1 = np.load("Y_all_area1_0.25.npy")
Y_all_area3 = np.load("Y_all_area3_0.25.npy")

In [ ]:
X_all = np.concatenate([Xp_all_area1, Xp_all_area3], axis=0)

In [ ]:
X_train1 = Xp_all_area1[:2400, :, :, :]
X_train1 = np.concatenate([X_train1, Xp_all_area3[:1000, :, :, :]])

X_test = Xp_all_area1[2400:, :, :, :]
X_test = np.concatenate([X_test, Xp_all_area3[1000:, :, :, :]])

In [ ]:
Y_train1 = Y_all_area1[:2400, :, :, :]
Y_train1 = np.concatenate([Y_train1, Y_all_area3[:1000, :, :, :]])

Y_test = Y_all_area1[2400:, :, :, :]
Y_test = np.concatenate([Y_test, Y_all_area3[1000:, :, :, :]])

In [ ]:
for i in range(X_train1.shape[0]):
    min_val = np.min(X_train1[i,:,:,:])           
    max_val = np.max(X_train1[i,:,:,:])
    X_train1[i,:,:,:] = (X_train1[i,:,:,:]- min_val)/(max_val - min_val)

for i in range(X_test.shape[0]):
    min_val = np.min(X_test[i,:,:,:])           
    max_val = np.max(X_test[i,:,:,:])
    X_test[i,:,:,:] = (X_test[i,:,:,:]- min_val)/(max_val - min_val)
    
    
# X_train1 = (X_train1 - min_val)/(max_val - min_val)
X_train1=np.reshape(X_train1,(3400,6400))

# X_test = (X_test - min_val)/(max_val - min_val)
X_test=np.reshape(X_test,(347,6400))

X_train,X_test1,Y_train,Y_test1=train_test_split(X_train1,Y_train1,test_size=0.00001,random_state=42,shuffle=True)

In [ ]:
#NN model-1

input_1 = Input(shape = (6400,), name = 'input_1')
x = Dense(4096, activation='relu')(input_1)


x = tf.reshape(x, [-1,32,32,4,1])


x = UpSampling3D((3, 3, 3))(x)


x = Conv3D(2,kernel_size = (13,13,2),activation='relu')(x)

x = Conv3D(4,kernel_size = (13,13,3),activation='relu')(x)

x = BatchNormalization()(x)

x = Conv3D(8,kernel_size = (9,9,2),activation='sigmoid')(x)

output = tf.reshape(x, [-1,128,128,16])


model = Model(inputs=input_1, outputs=output)
print(model.summary())

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0002,decay_steps=10000,decay_rate=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=opt, loss=CustomLoss_BCE(),metrics=['binary_accuracy'])
# #model.summary()

In [ ]:
#iteration  : 28 epochs done
batch_size = 32
epochs = 50
for n_epoch in range(epochs):
    print("Epoch: ", n_epoch)
    model.fit(X_train, Y_train, epochs=1, batch_size=batch_size)
    model.save_weights('model_norm_cae_Ts_CL_ep100_ArrData_woReg_AA13_epc30.h5')  